In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:100"

In [9]:
!nvidia-smi

Thu Feb 16 12:45:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.78       Driver Version: 512.78       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8     3W /  N/A |   2252MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import torch
print(torch.cuda.is_available())

True


In [4]:
# si solo queremos procesar el corpus (no codificarlo):
from load_data import Data

corpus = Data()
corpus.processing()
#corpus.index2tag
#dir(corpus)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\evasa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/1330 [00:00<?, ?ex/s]

  0%|          | 0/94 [00:00<?, ?ex/s]

  0%|          | 0/333 [00:00<?, ?ex/s]

In [4]:
from tokenize_and_encode_data import Tokenizer

corpus = Tokenizer()
corpus.processing() # función de load_data

# despues de esto: 0MiB /  6144MiB

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\evasa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


  0%|          | 0/1330 [00:00<?, ?ex/s]

  0%|          | 0/94 [00:00<?, ?ex/s]

  0%|          | 0/333 [00:00<?, ?ex/s]

In [4]:
texto = "Jack Sparrow aime New York !" 
corpus.ejemplo_tokenizado(texto)
corpus.ver_input_ids(texto)

,0,1,2,3,4,5,6,7,8
Tokens,<s>,▁Jack,▁Spar,row,▁aime,▁New,▁York,▁!,</s>
Input IDs,0,21763,37456,15555,91022,2356,5753,711,2


In [5]:
corpus.encode()
# después ded eso usa 1156MiB /  6144MiB de RAM la GPU

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-st

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [6]:
len(corpus.clara_encoded['train']['input_ids'][10])

100

In [7]:
corpus.clara_encoded['train']['input_ids'] # .shape

[[0, 40115, 39, 6, 5, 2],
 [0, 106, 60547, 177, 6, 5, 2],
 [0,
  106,
  5419,
  132443,
  1514,
  90994,
  150331,
  20866,
  30357,
  116751,
  294,
  105029,
  48952,
  1514,
  7338,
  48952,
  1514,
  8719,
  418,
  6,
  5,
  2],
 [0, 239, 79512, 309, 318, 6889, 146, 104211, 6, 5, 2],
 [0,
  2203,
  19559,
  102385,
  86425,
  7,
  22,
  88,
  5120,
  18265,
  20179,
  146,
  212662,
  8,
  6524,
  1883,
  6,
  5,
  2],
 [0, 87652, 3825, 162822, 69994, 7, 8, 537, 56, 6, 5, 2],
 [0, 3050, 161313, 7, 6, 5, 2],
 [0, 2861, 2],
 [0, 8, 21, 347, 5, 5035, 2],
 [0, 146, 104, 5, 1892, 2],
 [0, 8, 21, 808, 6, 5, 2],
 [0, 2994, 28237, 39, 6, 5, 2],
 [0, 201, 92759, 91, 5, 36, 6, 5, 2],
 [0, 305, 92759, 6, 5, 2],
 [0, 91, 5, 36, 6, 5, 2],
 [0, 190, 92759, 6, 5, 2],
 [0, 91, 5, 91, 6, 5, 2],
 [0, 3253, 849, 37165, 6, 5, 2],
 [0, 11653, 2],
 [0, 915, 5, 4541, 2],
 [0, 96, 5, 11653, 2],
 [0, 915, 5, 4541, 2],
 [0, 96, 5, 11653, 2],
 [0, 915, 5, 4541, 2],
 [0, 96, 6, 5, 2],
 [0, 1913, 28237, 7, 452

In [7]:
from transformers import TrainingArguments
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import Trainer

In [8]:
num_epochs = 10
batch_size = 6 # antes 12 en GPU de 16GB
logging_steps = len(corpus.clara_encoded["train"]) // batch_size
model_name_output = "/" #f"./results"
training_args = TrainingArguments(
    output_dir=model_name_output,
    log_level="error",
    num_train_epochs=num_epochs, 
    per_device_train_batch_size=batch_size, 
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch", 
    save_steps=1e6,
    weight_decay=0.01,
    disable_tqdm=False, 
    logging_steps=logging_steps,
    push_to_hub=False)

data_collator = DataCollatorForTokenClassification(corpus.tokenizer, max_length = 100, return_tensors='pt') #  truncation = True, dice unexpected argument
                                                                                                            # el padding ya lo hemos hecho antes en tokenize_and_aling_labels

def model_init():
    return (AutoModelForTokenClassification # XLMRobertaForTokenClassification
            .from_pretrained(corpus.model_name, config=corpus.config)
            .to(corpus.device))

trainer = Trainer(model_init=model_init,
                  args=training_args, 
                  data_collator=data_collator,
                  compute_metrics=corpus.compute_metrics,
                  train_dataset=corpus.clara_encoded["train"],
                  eval_dataset=corpus.clara_encoded["val"], 
                  tokenizer=corpus.tokenizer)

# despues de esto: 2252MiB /  6144MiB

In [10]:
trainer.train()

  0%|          | 0/2220 [00:00<?, ?it/s]

c:\Users\evasa\anaconda3\envs\book\lib\site-packages\transformers\tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


RuntimeError: CUDA out of memory. Tried to allocate 734.00 MiB (GPU 0; 6.00 GiB total capacity; 4.55 GiB already allocated; 0 bytes free; 4.72 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [14]:
import torch
torch.cuda.empty_cache()
torch.cuda.memory_allocated(device=0)

4889561600

In [13]:
torch.no_grad()

In [ ]:
#trainer.push_to_hub(commit_message="Training completed!")
#trainer.save_model("xlm-roberta-retrained-ner-fr")

In [ ]:
df = pd.DataFrame(trainer.state.log_history)[['epoch','loss' ,'eval_loss', 'eval_f1']]
df = df.rename(columns={"epoch":"Epoch","loss": "Training Loss", "eval_loss": "Validation Loss", "eval_f1":"F1"})
df['Epoch'] = df["Epoch"].apply(lambda x: round(x))
df['Training Loss'] = df["Training Loss"].ffill()
df[['Validation Loss', 'F1']] = df[['Validation Loss', 'F1']].bfill().ffill()
df.drop_duplicates()

In [ ]:
trainer.evaluate(clara_encoded["test"])


In [ ]:
outputs = trainer.predict(clara_encoded["test"])
outputs.metrics

In [ ]:
from seqeval.metrics import classification_report
y_pred, y_true = corpus.align_predictions(outputs.predictions,  outputs.label_ids)
print(classification_report(y_true, y_pred))